In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from pathlib import Path
import gensim
import string
from nltk.corpus import stopwords, wordnet
import nltk
from nltk.tokenize import word_tokenize
from gensim import corpora, models
from nltk.stem.wordnet import WordNetLemmatizer
import re

In [2]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [3]:
topic_modeling_path = Path('/home3/ntueconfbra5/Desktop/post_message_without_http.csv')
politifact_topic_training = pd.read_csv(topic_modeling_path)
politifact_topic_training

,page_id,page_name,post_id,post_type,post_name,post_message,post_caption,post_picture,post_link,post_description,...,post_created_time_CT,post_updated_time_CT,post_created_date_CT,post_updated_date_CT,post_created_time,post_updated_time,page_talking_about_count,domain,category,type
0,440106476051475,Freedom Daily,440106476051475_1308905149171599,link,Donald Trump Thinks Some Kids NEED a GOOD Old ...,Donald Trump Thinks Some Kids NEED a GOOD Ol' ...,freedomdaily.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://freedomdaily.com/donald-trump-thinks-ki...,A good 'Ol Whoopin!,...,2016-11-22T10:28:48Z,2016-12-10T19:02:11Z,2016-11-22,2016-12-10,2016-11-22T16:28:48Z,2016-12-11T01:02:11Z,203082.0,FreedomDaily.com,Media/News/Publishing,media
1,610045389164725,Liberal Daily,610045389164725_770596729776256,link,"""Grow A Nut"", Whoopi Doesn't Hold Back, Just T...","Whoopi is pissed off, just let loose on the sp...",liberalplug.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://liberalplug.com/2017/02/07/grow-nut-who...,NaN,...,2017-02-07T14:00:03Z,2017-04-03T18:43:49Z,2017-02-07,2017-04-03,2017-02-07T20:00:03Z,2017-04-03T23:43:49Z,165919.0,LiberalPlug.com,Media/News/Publishing,media
2,440106476051475,Freedom Daily,440106476051475_1138216752907107,link,"Home Invasion Thug Stabs 75-Year-Old Man, 80-Y...","Home Invasion Thug Stabs 75-Year-Old Man, 80-Y...",freedomdaily.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://freedomdaily.com/home-invasion-thug-sta...,"An intruder broke into a home in Monroe, Washi...",...,2016-05-01T16:30:00Z,2016-05-03T13:40:50Z,2016-05-01,2016-05-03,2016-05-01T21:30:00Z,2016-05-03T18:40:50Z,273614.0,FreedomDaily.com,Media/News/Publishing,media
3,440106476051475,Freedom Daily,440106476051475_1376156649113115,link,Donald Trump Thinks Some Kids NEED a GOOD Old ...,Donald Trump Thinks Some Kids NEED a GOOD Old ...,freedomdaily.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://freedomdaily.com/donald-trump-thinks-ki...,A good 'Ol Whoopin!,...,2017-01-15T11:03:18Z,2017-01-22T11:47:35Z,2017-01-15,2017-01-22,2017-01-15T17:03:18Z,2017-01-22T17:47:35Z,200739.0,FreedomDaily.com,Media/News/Publishing,media
4,610045389164725,Liberal Daily,610045389164725_758948130941116,link,Jimmy Carter Flew To DC For Inauguration...But...,This is how a man of HONOR and DIGNITY behaves...,liberalplug.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://liberalplug.com/2017/01/21/jimmy-carter...,NaN,...,2017-01-21T17:30:00Z,2017-01-26T20:19:55Z,2017-01-21,2017-01-26,2017-01-21T23:30:00Z,2017-01-27T02:19:55Z,171811.0,LiberalPlug.com,Media/News/Publishing,media
5,440106476051475,Freedom Daily,440106476051475_1279170668811714,link,Media RAGING After City's Biggest Police Union...,Media RAGING After City's Biggest Police Union...,freedomdaily.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://freedomdaily.com/media-raging-citys-big...,HAha! Liberal media is freaking out because Sh...,...,2016-10-24T10:24:08Z,2016-10-26T03:01:07Z,2016-10-24,2016-10-26,2016-10-24T15:24:08Z,2016-10-26T08:01:07Z,437482.0,FreedomDaily.com,Media/News/Publishing,media
6,610045389164725,Liberal Daily,610045389164725_755235044645758,link,Liz Warren Just Struck At Trump With A Bill Th...,Hero Senator Liz Warren just filed a bill that...,liberalplug.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://liberalplug.com/2017/01/15/liz-warren-j...,NaN,...,2017-01-15T13:21:00Z,2017-02-18T21:07:05Z,2017-01-15,2017-02-18,2017-01-15T19:21:00Z,2017-02-19T03:07:05Z,171811.0,LiberalPlug.com,Media/News/Publishing,media
7,610045389164725,Liberal Daily,610045389164725_774673936035202,link,"Congressman Who Called Trump ""Racist"" Refuses ...",A Dem congressman caught flak for calling Trum...,liberalplug.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://liberalplug.com/2017/02/14/congressman-...,NaN,...,2017-02-14T14:00:00Z,2017-02-21T04:58:28Z,2017-02-14,2017-02-21,2017-02-14T20:00:00Z,2017-02-21T10:58:28Z,16

In [213]:
total_post_message = []
lemmatizer = WordNetLemmatizer()
for i in politifact_topic_training['post_message']:
    tokens = word_tokenize(str(i))
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words = nltk.pos_tag(words)
    l = []
    for word, tag in words:
        wntag = get_wordnet_pos(tag)
        if wntag is None:
            lemma = lemmatizer.lemmatize(word)
            l.append(lemma)
        else:
            lemma = lemmatizer.lemmatize(word, pos=wntag) 
            l.append(lemma)        
    total_post_message.append(l)

In [214]:

dictionary = corpora.Dictionary(total_post_message)
corpus = [dictionary.doc2bow(text) for text in total_post_message]


In [215]:
tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]
model2 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=5, id2word=dictionary)
model2.print_topics(num_words=5)

[(0, '0.004*"go" + 0.004*"one" + 0.004*"take" + 0.004*"woman" + 0.004*"find"'),
 (1,
  '0.009*"please" + 0.008*"enjoy" + 0.008*"market" + 0.007*"share" + 0.007*"love"'),
 (2,
  '0.012*"illuminati" + 0.007*"false" + 0.007*"secret" + 0.007*"top" + 0.007*"video"'),
 (3,
  '0.010*"jim" + 0.008*"video" + 0.008*"obama" + 0.005*"government" + 0.005*"trump"'),
 (4,
  '0.021*"willie" + 0.019*"peter" + 0.019*"schiff" + 0.011*"global" + 0.010*"dollar"')]

In [216]:
model6 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=6, id2word=dictionary)
model6.print_topics(num_words=5)

[(0,
  '0.006*"ancient" + 0.005*"martial" + 0.005*"video" + 0.005*"human" + 0.004*"pyramid"'),
 (1,
  '0.011*"peter" + 0.011*"schiff" + 0.008*"dollar" + 0.008*"video" + 0.008*"illuminati"'),
 (2,
  '0.007*"nuclear" + 0.006*"time" + 0.005*"obama" + 0.004*"confession" + 0.004*"choice"'),
 (3,
  '0.006*"bank" + 0.005*"enjoy" + 0.004*"good" + 0.004*"list" + 0.004*"check"'),
 (4,
  '0.016*"jim" + 0.009*"collapse" + 0.007*"video" + 0.007*"economy" + 0.007*"government"'),
 (5,
  '0.021*"willie" + 0.011*"global" + 0.007*"false" + 0.005*"top" + 0.005*"finance"')]

In [217]:
model7 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=7, id2word=dictionary)
model7.print_topics(num_words=5)

[(0,
  '0.034*"willie" + 0.012*"dollar" + 0.009*"fwd" + 0.008*"china" + 0.008*"choice"'),
 (1,
  '0.005*"interest" + 0.005*"school" + 0.005*"currency" + 0.004*"rare" + 0.004*"go"'),
 (2,
  '0.012*"global" + 0.010*"government" + 0.009*"collapse" + 0.009*"video" + 0.008*"donaldtrump"'),
 (3,
  '0.015*"illuminati" + 0.008*"nuclear" + 0.008*"obama" + 0.007*"confession" + 0.006*"conspiracy"'),
 (4,
  '0.015*"peter" + 0.007*"economic" + 0.006*"obama" + 0.006*"false" + 0.005*"top"'),
 (5,
  '0.020*"schiff" + 0.009*"economy" + 0.008*"history" + 0.008*"human" + 0.006*"ancient"'),
 (6,
  '0.015*"jim" + 0.006*"market" + 0.005*"video" + 0.004*"finance" + 0.004*"russia"')]

In [218]:
model8 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=8, id2word=dictionary)
model8.print_topics(num_words=5)

[(0,
  '0.013*"nuclear" + 0.011*"human" + 0.009*"bank" + 0.008*"confession" + 0.008*"ancient"'),
 (1,
  '0.017*"illuminati" + 0.008*"conspiracy" + 0.006*"theory" + 0.006*"powerful" + 0.006*"reserve"'),
 (2,
  '0.021*"peter" + 0.021*"schiff" + 0.010*"obama" + 0.010*"economy" + 0.008*"market"'),
 (3,
  '0.005*"antarctica" + 0.004*"come" + 0.004*"picture" + 0.004*"area" + 0.004*"sound"'),
 (4,
  '0.017*"global" + 0.014*"finance" + 0.012*"crazy" + 0.012*"choice" + 0.012*"currency"'),
 (5,
  '0.026*"willie" + 0.010*"please" + 0.008*"enjoy" + 0.007*"hillary" + 0.007*"share"'),
 (6,
  '0.016*"jim" + 0.009*"dollar" + 0.009*"collapse" + 0.007*"economic" + 0.006*"obama"'),
 (7,
  '0.006*"video" + 0.005*"false" + 0.005*"police" + 0.004*"go" + 0.004*"people"')]

In [219]:
model15 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=15, id2word=dictionary)
model15.print_topics(num_words=5)

[(0,
  '0.030*"global" + 0.014*"currency" + 0.013*"warn" + 0.012*"china" + 0.012*"war"'),
 (1,
  '0.010*"obamaland" + 0.009*"weird" + 0.008*"poetic" + 0.007*"space" + 0.007*"health"'),
 (2,
  '0.037*"peter" + 0.009*"boom" + 0.008*"obama" + 0.008*"guess" + 0.007*"feed"'),
 (3,
  '0.028*"collapse" + 0.012*"choice" + 0.011*"multiple" + 0.011*"fwd" + 0.011*"quiz"'),
 (4,
  '0.028*"jim" + 0.017*"illuminati" + 0.010*"obama" + 0.008*"video" + 0.008*"bank"'),
 (5,
  '0.021*"dollar" + 0.016*"government" + 0.014*"donaldtrump" + 0.009*"mind" + 0.009*"unreal"'),
 (6,
  '0.051*"schiff" + 0.025*"economic" + 0.022*"market" + 0.014*"build" + 0.013*"pyramid"'),
 (7,
  '0.014*"mass" + 0.012*"google" + 0.010*"missile" + 0.008*"file" + 0.008*"iran"'),
 (8,
  '0.013*"conspiracy" + 0.013*"good" + 0.010*"luck" + 0.009*"area" + 0.008*"disturb"'),
 (9,
  '0.014*"top" + 0.012*"ancient" + 0.008*"arm" + 0.007*"society" + 0.007*"isi"'),
 (10,
  '0.015*"martial" + 0.010*"great" + 0.009*"deal" + 0.009*"event" + 0.00

# 0.01 sampling

In [13]:
total_post_message = []
politifact_topic_training = politifact_with_fake_domain[politifact_with_fake_domain.post_message.notna()]
sampling = politifact_topic_training.sample(frac=0.01)
sampling

,page_id,page_name,post_id,post_type,post_name,post_message,post_caption,post_picture,post_link,post_description,...,post_created_time_CT,post_updated_time_CT,post_created_date_CT,post_updated_date_CT,post_created_time,post_updated_time,page_talking_about_count,domain,category,type
117544,702815073143116,Rebirth of Freedom,702815073143116_987919877965966,link,Teen's Eyes Do Horrifying 'Trick' Every 5 Mins...,A gorgeous 17-year-old has been forced to hide...,madworldnews.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://madworldnews.com/teens-eyes-trick-5-mins/,A gorgeous 17-year-old has been forced to hide...,...,2016-03-16T20:45:00Z,2016-03-17T12:48:20Z,2016-03-16,2016-03-17,2016-03-17T01:45:00Z,2016-03-17T17:48:20Z,16482.0,MadWorldNews.com,Publisher,media
110169,158924294183807,Allen West Republic,158924294183807_1053798581363036,link,Academia sets another new world record in shar...,Academia sets another new world record in shar...,bb4sp.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://bb4sp.com/professors-outraged-over-pic-...,You have GOT to be kidding me!,...,2016-04-14T10:12:11Z,2016-04-14T10:23:09Z,2016-04-14,2016-04-14,2016-04-14T15:12:11Z,2016-04-14T15:23:09Z,465427.0,BB4SP.com,Public Figure,figure
107587,312531385450925,The Last Great Stand,312531385450925_1252136188157102,link,More Fake News! Why Do They Refuse to be Hones...,More Fake News! Why Do They Refuse to be Hones...,beforeitsnews.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://buff.ly/2kybrAn,It’s becoming more and more clear with each pa...,...,2017-02-12T06:00:53Z,2017-02-12T15:36:55Z,2017-02-12,2017-02-12,2017-02-12T12:00:53Z,2017-02-12T21:36:55Z,3821.0,BeforeItsNews.com,Politician,media
96111,149817218374125,Girls Just Wanna Have Guns,149817218374125_1156245247731312,link,WATCH: Political ‘Sugar-Daddy’ Says Trump’s HU...,If this is TRUE I hope they're ready for a rev...,clashdaily.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://ow.ly/yYxe303SCVv,Bernie fans were right about Big Money buying ...,...,2016-09-04T09:25:25Z,2016-09-05T00:04:16Z,2016-09-04,2016-09-05,2016-09-04T14:25:25Z,2016-09-05T05:04:16Z,29869.0,ClashDaily.com,Cause,group
29535,158924294183807,Allen West Republic,158924294183807_1031452056931022,link,Muslim Students Demand Special Treatment… Univ...,Muslim Students Demand Special Treatment… Univ...,bb4sp.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://bb4sp.com/muslim-students-demand-specia...,After a group of Muslim college students began...,...,2016-03-18T02:11:00Z,NaN,2016-03-18,NaN,2016-03-18T07:11:00Z,NaN,NaN,BB4SP.com,Public Figure,figure
58003,513813158657249,Mad World News,513813158657249_888943147810913,link,"Tyrant Goes BERSERK At Walmart, White Trash Mo...","More often than not, Walmart offers a dysfunct...",madworldnews.com,https://fbexternal-a.akamaihd.net/safe_image.p...,http://madworldnews.com/tyrant-goes-berserk-wa...,"More often than not, Walmart offers a dysfunct...",...,2015-05-28T14:00:31Z,2015-05-30T09:21:47Z,2015-05-28,2015-05-30,2015-05-28T19:00:31Z,2015-05-30T14:21:47Z,149702.0,MadWorldNews.com,News/Media Website,media
75612,249903348378912,Obama Makes Me Puke,249903348378912_869994043036503,link,GUESS WHO GIVES THE 'ALLAH IS THE ONLY GOD' SI...,Make sure you go all the way to the end to see...,clashdaily.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://clashdaily.com/2015/03/guess-who-gives-...,This is an album of pictures of a Muslim hand ...,...,2015-03-13T10:59:40Z,2015-03-13T10:59:40Z,2015-03-13,2015-03-13,2015-03-13T15:59:40Z,2015-03-13T15:59:40Z,9203.0,ClashDaily.com,Just For Fun,group
114776,312531385450925,The Last Great Stand,312531385450925_895260967177961,link,Jade Helm: Do You Realize The Nation May Never...,Jade Helm: Do You Realize The Nation May Never...,beforeitsnews.com,https://fbexternal-a.akamaihd.net/safe_image.p...,http://beforeitsnews.com/politics/2015/08/jade...,FOR MORE NEWS BY VOICE OF REASON CLICK HERE! B...,...,20

In [14]:
for i in sampling['post_message']:
    tokens = word_tokenize(i)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    total_post_message.append(words)

In [15]:

dictionary = corpora.Dictionary(total_post_message)
corpus = [dictionary.doc2bow(text) for text in total_post_message]


## into different number of topic

In [16]:
tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]

model2 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=5, id2word=dictionary)
model2.print_topics(num_words=5)

[(0,
  '0.003*"http" + 0.002*"enjoy" + 0.002*"hillary" + 0.001*"good" + 0.001*"right"'),
 (1,
  '0.005*"http" + 0.002*"get" + 0.002*"obama" + 0.002*"man" + 0.001*"time"'),
 (2,
  '0.003*"http" + 0.002*"one" + 0.002*"video" + 0.002*"woman" + 0.001*"girl"'),
 (3,
  '0.003*"obama" + 0.003*"love" + 0.003*"http" + 0.003*"think" + 0.002*"trump"'),
 (4,
  '0.003*"http" + 0.002*"obama" + 0.002*"thoughts" + 0.002*"video" + 0.002*"hillary"')]

In [17]:
model6 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=6, id2word=dictionary)
model6.print_topics(num_words=5)

[(0,
  '0.003*"http" + 0.002*"one" + 0.002*"thoughts" + 0.001*"could" + 0.001*"woman"'),
 (1,
  '0.003*"http" + 0.002*"get" + 0.002*"never" + 0.002*"mother" + 0.002*"obama"'),
 (2,
  '0.003*"http" + 0.003*"obama" + 0.002*"love" + 0.002*"please" + 0.002*"hillary"'),
 (3,
  '0.003*"http" + 0.002*"video" + 0.002*"obama" + 0.002*"share" + 0.002*"quiz"'),
 (4,
  '0.003*"http" + 0.003*"trump" + 0.002*"video" + 0.002*"obama" + 0.002*"hillary"'),
 (5,
  '0.005*"http" + 0.002*"think" + 0.002*"hillary" + 0.002*"obama" + 0.002*"check"')]

In [18]:
model7 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=7, id2word=dictionary)
model7.print_topics(num_words=5)

[(0,
  '0.003*"http" + 0.002*"kiddies" + 0.002*"quiz" + 0.002*"multiple" + 0.002*"choice"'),
 (1,
  '0.003*"http" + 0.002*"video" + 0.002*"america" + 0.002*"get" + 0.002*"obama"'),
 (2,
  '0.003*"http" + 0.003*"love" + 0.002*"share" + 0.002*"video" + 0.002*"trump"'),
 (3,
  '0.003*"http" + 0.002*"hillary" + 0.002*"right" + 0.002*"man" + 0.002*"trump"'),
 (4,
  '0.003*"http" + 0.003*"obama" + 0.002*"think" + 0.002*"something" + 0.002*"would"'),
 (5,
  '0.005*"http" + 0.002*"one" + 0.002*"said" + 0.002*"hillary" + 0.002*"could"'),
 (6,
  '0.003*"http" + 0.002*"email" + 0.002*"obama" + 0.002*"hillary" + 0.002*"awesome"')]

In [19]:
model8 =models.ldamodel.LdaModel(tfidf_corpus, num_topics=8, id2word=dictionary)
model8.print_topics(num_words=5)

[(0,
  '0.003*"http" + 0.002*"mother" + 0.002*"obama" + 0.002*"hillary" + 0.002*"interesting"'),
 (1,
  '0.003*"http" + 0.002*"trump" + 0.002*"think" + 0.002*"enjoy" + 0.002*"hillary"'),
 (2,
  '0.003*"http" + 0.002*"get" + 0.002*"man" + 0.002*"hillary" + 0.002*"everyone"'),
 (3,
  '0.006*"http" + 0.002*"brutal" + 0.002*"love" + 0.002*"would" + 0.002*"latest"'),
 (4,
  '0.003*"http" + 0.002*"obama" + 0.002*"hillary" + 0.002*"love" + 0.002*"right"'),
 (5,
  '0.003*"obama" + 0.003*"http" + 0.003*"enjoy" + 0.002*"never" + 0.002*"trump"'),
 (6,
  '0.003*"video" + 0.002*"http" + 0.002*"obama" + 0.002*"donaldtrump" + 0.002*"like"'),
 (7,
  '0.003*"http" + 0.002*"well" + 0.002*"one" + 0.002*"hillary" + 0.002*"get"')]